# Preprocessing: BJH024


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data. Skipped channel name in regions we aren't interested in. I think its fine.

However, the skipped channels are messing up the function to find its pairs, which leaves a bunch of regions not marked as bad but also not bipolar rereferenced. Need to redo that function to work with missing channels


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH024'
raw_data_dir = f"/media/brooke/T7/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /media/brooke/T7/pacman/raw_data/BJH024/ieeg/BJH024_notched_filtered_clean_ieeg.fif...


    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /media/brooke/T7/pacman/raw_data/BJH024/ieeg/BJH024_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 4171999 =   1043.000 ...  2085.999 secs
Ready.
Opening raw data file /media/brooke/T7/pacman/raw_data/BJH024/ieeg/BJH024_notched_filtered_clean_ieeg-2.fif...
    Range : 4172000 ... 4365199 =   2086.000 ...  2182.599 secs
Ready.
Opening raw data file /media/brooke/T7/pacman/raw_data/BJH024/ieeg/BJH024_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /media/brooke/T7/pacman/raw_data/BJH024/ieeg/BJH024_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 4171999 =   1043.000 ...  2085.999 secs
Ready.
Opening raw data file /media/brooke/T7/pacman/raw_data/BJH024/ieeg/BJH024_raw_clean_ieeg-2.fif...
    Range : 4172000 ... 4365199 =   2086.000 ...  2182.599 secs
Ready.


In [16]:
raw_clean_fif.info['ch_names']

['AL1',
 'AL2',
 'AL3',
 'AL5',
 'REF1',
 'REF2',
 'AL6',
 'AL7',
 'AL8',
 'AL9',
 'AL11',
 'AL12',
 'AL13',
 'AL14',
 'BL1',
 'BL2',
 'BL3',
 'BL4',
 'BL5',
 'BL6',
 'BL7',
 'BL9',
 'BL10',
 'BL11',
 'BL12',
 'BL14',
 'BL15',
 'BL16',
 'CL1',
 'CL2',
 'CL7',
 'CL8',
 'CL9',
 'CL10',
 'CL12',
 'CL13',
 'CL14',
 'CL15',
 'CL16',
 'DL1',
 'DL2',
 'DL3',
 'DL11',
 'DL12',
 'DL13',
 'DL14',
 'DL15',
 'DL16',
 'EL1',
 'EL2',
 'EL3',
 'EL4',
 'EL5',
 'EL6',
 'EL7',
 'EL8',
 'EL9',
 'EL10',
 'EL11',
 'EL12',
 'FL1',
 'FL2',
 'FL3',
 'FL4',
 'FL5',
 'FL6',
 'FL7',
 'FL8',
 'FL9',
 'FL10',
 'GL1',
 'GL2',
 'GL3',
 'GL4',
 'GL5',
 'GL6',
 'GL7',
 'GL8',
 'GL9',
 'GL10',
 'GL11',
 'GL12',
 'GL13',
 'GL14',
 'HL1',
 'HL2',
 'HL3',
 'HL4',
 'HL5',
 'HL6',
 'HL7',
 'HL8',
 'HL9',
 'HL10',
 'HL11',
 'HL12',
 'HL13',
 'HL14',
 'IL1',
 'IL2',
 'IL3',
 'IL4',
 'IL5',
 'IL6',
 'IL11',
 'IL12',
 'IL13',
 'IL14',
 'JL1',
 'JL2',
 'JL3',
 'JL4',
 'JL5',
 'JL6',
 'JL7',
 'JL8',
 'JL9',
 'JL10',
 'JL11',
 'JL

## Bipolar Rereferencing

In [ ]:
bad_numbering = {'AL5': 'AL4',
                 'AL6': 'AL5',
                 'AL7': 'AL6',
                 'AL8': 'AL7',
                 'AL9': 'AL8',
                 'AL11': 'AL9',
                 'AL12': 'AL10',
                 'AL13': 'AL11',
                 'AL14': 'AL12',
                 'AL14': 'AL13'}


mne.channels.rename_channels(filtered_clean_fif.info, {' AR13': 'AR13'})
mne.channels.rename_channels(raw_clean_fif.info, {' AR13': 'AR13'})

In [33]:
other_in_probe

['AL1',
 'AL2',
 'AL3',
 'AL5',
 'AL6',
 'AL7',
 'AL8',
 'AL9',
 'AL11',
 'AL12',
 'AL13',
 'AL14']

In [6]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI' or next_ch not in labels:
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [7]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

AL1 AL2
AL2 AL3
AL3 AL5
AL5 AL6
REF1 noref
REF2 noref
AL6 AL7
AL7 AL8
AL8 AL9
AL9 AL11
AL11 AL12
BL1 BL2
BL2 BL3
BL3 BL4
BL4 BL5
BL5 BL6
BL6 BL7
BL7 BL9
BL9 BL10
BL10 BL11
BL11 BL12
BL12 BL14
CL1 CL2
CL2 CL7
CL7 CL8
CL8 CL9
CL9 CL10
DL1 DL2
DL2 DL3
EL1 EL4
EL2 noref
EL3 noref
EL4 EL5
EL5 EL6
EL6 EL7
EL7 EL8
EL8 EL9
EL9 EL10
EL10 EL11
EL11 EL12
FL1 FL2
FL2 FL3
FL3 FL4
FL4 FL5
FL5 FL6
FL6 FL7
FL7 FL9
FL8 noref
FL9 FL10
GL1 noref
GL2 noref
GL3 GL4
GL4 GL5
GL5 GL6
GL6 GL7
GL7 GL8
GL8 GL9
GL9 GL10
GL10 GL11
GL11 GL12
GL12 GL13
GL13 GL14
HL1 noref
HL2 noref
HL3 noref
HL4 noref
HL5 HL6
HL6 HL7
HL7 HL8
HL8 HL9
HL9 HL10
HL10 HL11
HL11 HL12
HL12 HL13
HL13 HL14
IL1 IL2
IL2 IL3
IL3 IL4
IL4 IL5
IL5 IL6
JL1 JL2
JL2 JL3
JL3 JL4
JL4 JL5
JL5 JL6
JL6 JL7
JL7 JL8
JL8 JL9
JL9 JL10
JL10 JL11
JL11 JL12
AR1 AR2
AR2 AR3
AR3 AR4
AR4 AR5
AR5 AR6
AR6 AR7
AR7 AR8
AR8 AR9
AR9 AR10
AR10 AR11
AR11 AR13
BR1 BR2
BR2 BR3
BR3 BR4
BR4 BR5
BR5 BR6
BR6 BR7
BR7 BR8
BR8 BR9
BR9 BR10
BR10 BR11
BR11 BR12
BR12 BR13
BR13 BR14
BR

In [8]:
ch = 'AL13'
if ch in remove or ch == 'STI':
        print(f"{ch} noref")
else:
    print(ch)
    pair = find_bipolar_pair(ch, labels, remove)
    if pair:
        anode.append(ch)
        cathode.append(pair)
        pairs.append((ch, pair))
        pairs_map[ch] = f"{ch}-{pair}"
        pairs_name.append(f"{ch}-{pair}")
        print(ch, pair)

AL13


In [9]:
remove

['REF1',
 'REF2',
 'EKG1',
 'EKG2',
 'EL2',
 'EL3',
 'GL1',
 'GL2',
 'HL4',
 'HL3',
 'HL2',
 'ER7',
 'HL1',
 'FL8',
 'IR1',
 'IR2',
 'IR3',
 'HR1',
 'HR2',
 'FP1',
 'F3',
 'C3',
 'P3',
 'O1',
 'FP2',
 'F4',
 'C4',
 'P4',
 'O2',
 'F7',
 'T7',
 'P7',
 'F8',
 'T8',
 'P8',
 'F9',
 'F10',
 'FPZ',
 'FZ',
 'CZ',
 'PZ',
 'OZ',
 'EKG1',
 'EKG2']

In [10]:
bipolar_pair = None
probe, num = probe_and_num(ch)
other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
for i in range(len(other_in_probe)-num):
    next_ch = probe + str(num+i+1)
    if next_ch in remove or next_ch == 'STI' or next_ch not in labels:
        continue
    else:
        bipolar_pair = next_ch
        break


In [11]:
other_in_probe

['AL1',
 'AL2',
 'AL3',
 'AL5',
 'AL6',
 'AL7',
 'AL8',
 'AL9',
 'AL11',
 'AL12',
 'AL13',
 'AL14']

## Epoching the data 

### Onset

In [12]:
# bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
bp_filtered_fif.info['ch_names']

NameError: name 'bp_filtered_fif' is not defined

In [5]:


events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)

484 events found on stim channel STI
Event IDs: [0 1 2]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated


In [9]:
bp_filtered_fif.info['bads']

['C3',
 'C4',
 'CZ',
 'EKG1',
 'EKG2',
 'EL2',
 'EL3',
 'ER7',
 'F10',
 'F3',
 'F4',
 'F7',
 'F8',
 'F9',
 'FL8',
 'FP1',
 'FP2',
 'FPZ',
 'FZ',
 'GL1',
 'GL2',
 'HL1',
 'HL2',
 'HL3',
 'HL4',
 'HR1',
 'HR2',
 'IR1',
 'IR2',
 'IR3',
 'O1',
 'O2',
 'OZ',
 'P3',
 'P4',
 'P7',
 'P8',
 'PZ',
 'REF1',
 'REF2',
 'T7',
 'T8']

In [12]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif...
    Range : 0 ... 2263999 =      0.000 ...  1131.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data-1.fif...
    Range : 2264000 ... 4365199 =   1132.000 ...  2182.599 secs
Ready.


/tmp/ipykernel_59279/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


484 events found on stim channel STI
Event IDs: [0 1 2]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_59279/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...


### Trial End

In [19]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1] # remove last event

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif...
    Range : 0 ... 2263999 =      0.000 ...  1131.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data-1.fif...
    Range : 2264000 ... 4365199 =   1132.000 ...  2182.599 secs
Ready.


/tmp/ipykernel_59279/1462480556.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


484 events found on stim channel STI
Event IDs: [0 1 2]
Not setting metadata
241 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.


/tmp/ipykernel_59279/1462480556.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 121 events and 20001 original time points ...
Overwriting existing file.
Loading data for 120 events and 20001 original time points ...


### Last Away

In [4]:
## Load Behavioral Data ##
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,1,1,54400,0,1
1,2,4,63800,0,1
2,3,15,82300,0,1
3,4,11,95300,0,1
4,5,6,114000,0,1
...,...,...,...,...,...
208,233,4,4177600,0,1
209,234,12,4189000,0,1
210,235,9,4201000,0,1
211,237,16,4236000,0,1


In [5]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [6]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif...
    Range : 0 ... 2263999 =      0.000 ...  1131.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data-1.fif...


/tmp/ipykernel_6396/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


    Range : 2264000 ... 4365199 =   1132.000 ...  2182.599 secs
Ready.
Not setting metadata
213 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...


/tmp/ipykernel_6396/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Splitting into 2 parts
Loading data for 107 events and 20001 original time points ...
Loading data for 106 events and 20001 original time points ...


### First Dot

In [7]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,4,59600,0,1
1,3,15,76400,0,1
2,4,11,93300,0,1
3,5,6,104900,0,1
4,7,18,186800,0,1
...,...,...,...,...,...
222,235,9,4198900,0,1
223,236,1,4209600,0,1
224,237,16,4234000,0,1
225,238,8,4243800,0,1


In [8]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [9]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif...
    Range : 0 ... 2263999 =      0.000 ...  1131.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data-1.fif...
    Range : 2264000 ... 4365199 =   1132.000 ...  2182.599 secs
Ready.
Not setting metadata
227 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 114 events and 18001 original time points ...


/tmp/ipykernel_6396/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6396/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 113 events and 18001 original time points ...


## First Move

In [10]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,1,1,54400,0,1
1,2,4,59500,0,1
2,3,15,74100,0,1
3,4,11,92600,0,1
4,5,6,104800,0,1
...,...,...,...,...,...
233,235,9,4198100,0,1
234,236,1,4208800,0,1
235,237,16,4233800,0,1
236,238,8,4242900,0,1


In [11]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [12]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif...
    Range : 0 ... 2263999 =      0.000 ...  1131.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data-1.fif...
    Range : 2264000 ... 4365199 =   1132.000 ...  2182.599 secs
Ready.
Not setting metadata
238 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...


/tmp/ipykernel_6396/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6396/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Splitting into 2 parts
Loading data for 119 events and 20001 original time points ...
Loading data for 119 events and 20001 original time points ...


## Ghost Attack

In [13]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,5,6,113800,0,1
1,9,12,216200,0,1
2,10,10,228700,0,1
3,24,5,413000,0,1
4,29,15,463600,0,1
5,31,4,485800,0,1
6,35,2,527900,0,1
7,42,13,618000,0,1
8,43,11,630600,0,1
9,47,16,676100,0,1


In [14]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [15]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif...
    Range : 0 ... 2263999 =      0.000 ...  1131.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data-1.fif...
    Range : 2264000 ... 4365199 =   1132.000 ...  2182.599 secs
Ready.
Not setting metadata
58 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_6396/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6396/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 1 events and 20001 original time points ...
Loading data for 58 events and 20001 original time points ...
